In [1]:
import pandas as pd
import os
import sys
import json
import numpy as np
import tensorflow as tf
import keras.layers as L
import keras.activations as A
import keras.optimizers as O
import keras.losses as Loss
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras import backend as KB
from keras.callbacks import Callback
import time

/home/armughan/anaconda3/envs/py3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
ROOTDIR=os.path.abspath("../../")
sys.path.append(ROOTDIR)

In [3]:
from Preprocessing.Preprocessor import preprocessor
from data_utils.keras_generator import Generator
from utils.losses import weighted_cross_entropy_loss,f1_score_metric
from utils.callbacks import get_tensorboard_callback,get_checkpoint_call_back

In [4]:
label_encoder_dir=os.path.join(ROOTDIR,"Preprocessing")
data_dir=os.path.join(ROOTDIR,"Data")

In [5]:
P=preprocessor(label_encoder_dir)
p_config={'labels':6,
        'weightage':[1,1,1,1,1,1],
        'data_directory':data_dir,
        'file_batch_size':10,
        'format_':'h5',
        'ex_batch_size':10,
        'file_examples':25000,
        'Preprocessor':P,
        'static_weightage':False}

In [6]:
G=Generator(config=p_config)

In [11]:
class FC_Model():
    def __init__(self,config):
        self.config=config
        self.Neurons=config['Hidden']
        self.activation=config['Activation']
        self.output=config['Output']
        self.input_dim=config['Input_shape']
        self.optimizer=config['Optimizer']
        self.model=self.make_model()
        self.loss(config['loss'])
        
    def make_model(self):
        x=L.Input(shape=(self.input_dim,))
        inp=x
        for i in self.Neurons:
            x=L.Dense(i)(x)
            x=L.BatchNormalization()(x)
            x=L.Activation("relu")(x)
        x=L.Dense(self.output,activation='linear')(x)
        return Model(inp,x)
    
    def loss(self,loss):
        f1=f1_score_metric(self.output)
        self.model.compile(loss=loss, optimizer=self.optimizer,metrics=["acc",f1])
        
    def fit(self,generator):
        tensorboard=get_tensorboard_callback(logdir='../logs/fc_net')
        checkpoint=get_checkpoint_call_back("../checkpoints","fc_net_chkpoint_",period=5)
        
        self.model.fit_generator(generator= generator, 
                    steps_per_epoch  = len(generator), 
                    epochs           = self.config['epochs'], 
                    verbose          = 1,
#                     validation_data  = valid_batch,
#                     validation_steps = len(valid_batch),
                    max_queue_size   = 3)        

In [12]:
n_config={'Hidden':[200],
          'Activation':'relu',
          'Output':p_config['labels'],
          'Optimizer':O.Adam(lr=0.0001),
          'Input_shape':477,'epochs':1 ,
          'loss':weighted_cross_entropy_loss}
model=FC_Model(n_config)

In [13]:
model.fit(generator=G)

Epoch 1/1


/home/armughan/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


ValueError: setting an array element with a sequence.